In [370]:
    # import the libraies that will be using
import numpy as np
import pandas as pd
import vaex 
import glob

# about how to use vaex and run this jupyter note book
# please install vaex with Anaconda or with regular python intall
# in Anaconda cmd: conda install -c conda-forge vaex
# in regular cmd(python installed): pip install vaex

In [371]:
# open the hdf5 file with vaex, todealing with the large dataset, should convert the csv to hdf5
# caution with Windows operating system, python 3.8 with Windows may cause bugs when dealing with vaex

# After debugging the data, sold_at seems have some relation to expired_at. I don't know why sold_at can't filter out None, it will show error.
# Also, every time I try to catch specific information in the sold_at, such as find which row contains 2019 or 2020, it still shows a lot of None on sold_at column.
# It should output rows that is with stings contains 2019 or 2020 in sold_at but not with None, it seems sold_at column do have some problems.
# I do find when I search sold_at with contains 2019 or 2020, the date such as expired_at, created_at, or updated_on do have some relation to it, even sold_at still shows None.

    # road the hdf5 file
df = vaex.open('listings_hdf5.hdf5')

# to convert csv to hdf5 is to type in cmd with : vaex convert file gaia-dr1.fits gaia-dr1.hdf5



In [372]:
# testing the is_rental correctness
#df['is_rental']
#(df.is_rental.values == 'f').sum() # 3508281
#(df.is_rental.values == 't').sum() # 406839
#(df.is_rental.values != 'f').sum() # 406968
#(df.is_rental.values != 't').sum() # 3508410


In [373]:
df.describe()

,address,municipality,province,postal_code,geocoding_type_id,id,mls,price,description,bedrooms,...,url_slug,square_footage,kitchen_count,is_rental,expired_at,sold_price,sold_at,thumbnails_count,address_id,address_ids
dtype,str,str,str,str,float64,int64,str,float64,str,float64,...,str,str,float64,str,str,float64,str,float64,float64,str
count,3915249,3914119,3909013,3858369,3505731,3915249,3915249,3913763,3907421,3690058,...,3915249,2578383,2974091,3915120,3587429,691112,613900,1503831,3849194,3858615
NA,0,1130,6236,56880,409518,0,0,1486,7828,225191,...,0,1336866,941158,129,327820,3224137,3301349,2411418,66055,56634
mean,--,--,--,--,22.882743142585667,5336143.436108917,--,581522.0258825023,--,3.0052253921212078,...,--,--,0.9920829591293608,--,--,483661.85834939935,--,21.069484536493796,1206016.7423920436,--
std,--,--,--,--,1.76213,1.1308e+06,--,1.08284e+06,--,1.52657,...,--,--,0.390108,--,--,552604,--,12.3117,795788,--
min,--,--,--,--,1,1,--,0,--,0,...,--,--,0,--,--,0,--,0,9431,--
max,--,--,--,--,27,7294048,--,1e+09,--,1593,...,--,--,11,--,--,4.625e+07,--,100,2.76438e+06,--


In [360]:
    # testing area
#ttt = df[df['sold_price'] > df['price']*3]
#ttt.head(5)

In [382]:
    # analyze and filter out useless information, make it more easy for data analysis and higher correctness.

    # lines that maybe useful
#(df_testna.municipality.isna()).sum()
#(df.municipality.isna()).sum()
#df = df[df['province'].isna() == False]
#df = df[df['province_same'].isna() == False]
#df = df[df['municipality'].isna() == False]
#df = df[df['municipality_same'].isna() == False]
#(df_testna.municipality.isna()).sum()
#(df.municipality.isna()).sum()

    # filter out the null value from several columns
df = df[df['updated_on'].isna() == False]
df = df[df['is_rental'].isna() == False]
    # not using isna() for postal code, because it will cause 2019 data with index error
df = df[df['postal_code'].str.len()>5]
df = df[df['sold_price'].isna() == False]
df = df[df['price'].isna() == False]
    # remove na and also which house type is not house or condo
df = df[df['style_id'].isna() == False]
df = df[df['style_id'] != 24]
df = df[df['style_id'] != 111]

    # filter out the single value of year and month
df['updated_on'].str.replace(' ', '-')
df['time'] = df['updated_on'].str.slice(start=0, stop=7)
df['year'] = df['time'].str.slice(start=0, stop=4)
df = df[df['year'].str.isdigit()]
df = df[df['year'].str.len()==4]
df['month'] = df['time'].str.slice(start=5, stop=7)
    
    # ignore the difference with upper and lower letter and export useful information with regions and postal code
#df['municipality_same'] = df.municipality.str.lower()
df['province_same'] = df.province.str.lower()
df['region_territory'] = df['postal_code'].str.slice(start=0, stop=1)
df['region_territory'] = df['region_territory'].str.upper()
df = df[df['region_territory'].str.isalpha()]
df['region_place'] = df['postal_code'].str.slice(start=0, stop=3)
df['region_place'] = df['region_place'].str.upper()

    # using number of kitchens, parking slots, bedrooms, and bathrooms to find out the size of the house
    # 4 data columns have high data completion
    # the data of square_footage has over 1336866 missing data it may impact the result
    # it is also possible to use square_footage though 
df['home_size'] = df['kitchen_count'] + df['parking_spaces'] + df['bedrooms'] + df['bathrooms']

    # it contains 82% of data with NA value, is a risk to filter it out, need to do it carefully
    # sold_at data do has some problems, most of it only shows None in the column
#df = df[df['sold_at'].isna() == False]

In [383]:
    # columns show which is condo or house
df['condo'] = ((df['style_id'] < 108) & (df['style_id'] > 104))
df['house'] = ((df['style_id'] > 107) | (df['style_id'] < 104))
df['home_type'] = df.condo.apply(lambda s: 'condo' if s==True else 'house')


In [384]:
    # filter out useless columns
df_clean = df.drop(['geocoding_type_id','id','mls','url','search','broker_id','the_geom','provider_id','virtual_tour_url'
                   ,'web_bug_url', 'quad_tree', 'title', 'real_estate_board_id', 'bedrooms_partial', 'bathrooms_partial', 'source_id'
                   , 'email', 'listing_images_count'])

In [385]:
    # show whether the deal is in Sep. Use it if only want the Sep. deals.
df_clean['is_sep'] = df_clean['month'] == '09'


In [386]:
    # separate rent and sell only houses, not yet consider the house for both
df_rent = df_clean[df_clean['is_rental'] == 't']
df_sell = df_clean[df_clean['is_rental'] == 'f']

In [387]:
    # find the total numbers of each deals that is sold price > list price and list price > sold price
    # earn_passive shows the deal higher than expected, earn_negative shows the deal lower than expected
df_sell['earn'] = df_sell['sold_price'] - df_sell['price']

df_sell['earn_passive'] = df_sell['earn'] > 0
df_sell['earn_negative'] = df_sell['earn'] <= 0

#df_sell

In [388]:
    # this is for house only for sell
    # setting the condition to filter out noise data.
    # need a better improve for this part with a more detailed house price knowledge.
df_sell_res = df_sell[(df_sell['year'] == '2020')|(df_sell['year'] == '2019')]
df_sell_res = df_sell_res[df_sell_res['sold_price']>40000]
df_sell_res = df_sell_res[df_sell_res['sold_price']<20000000]
#df_sell_res = df_sell_res[df_sell_res['sold_price']<df_sell_res['price']*3]
df_sell_res = df_sell_res[df_sell_res['price']>40000]
df_sell_res = df_sell_res[df_sell_res['price']<20000000]

In [389]:
df_sell_res.head(1)

#,address,municipality,province,postal_code,price,description,bedrooms,bathrooms,status_id,style_id,created_at,updated_on,extras,address_components,parking_spaces,has_garage,has_locker,url_slug,square_footage,kitchen_count,is_rental,expired_at,sold_price,sold_at,thumbnails_count,address_id,address_ids,time,year,month,province_same,region_territory,region_place,home_size,condo,house,home_type,is_sep,earn,earn_passive,earn_negative
0,"3 Geranium Crt, Toronto, ON, M2M0A2",Toronto,ON,M2M0A2,2.388e+06,'Stunning Executive Custom Built Home On Quiet C...,4,5,3,101,2020-03-10 08:45:40.479724-04,2020-04-16 16:45:45.766633,"'{""lot"": {""unit"": ""feet"", ""depth"": 103, ""front"":...","'{""postal"":""M2M0A2"",""street"":""3 Geranium Crt"",""s...",6,t,f,3-geranium-crt-toronto-on,"[3500,5001)",1,f,2020-04-07 00:00:00-04,2.2e+06,None,20,1.60803e+06,"{2875,1,2,8135}",2020-04,2020,04,on,M,M2M,16,False,True,house,False,-188000,False,True


In [391]:
    # the result grouping with place, year, and home type
df_res1 = df_sell_res.groupby(['region_place','year','home_type'],
              agg={'mean_sold': vaex.agg.mean('sold_price'),
                   'mean_price': vaex.agg.mean('price'),
                   'mean_rank': vaex.agg.mean('thumbnails_count'),
                   'mean_size': vaex.agg.mean('home_size'),
                   'min_sold': vaex.agg.min('sold_price'),
                   'min_price': vaex.agg.min('price'),
                   'max_size': vaex.agg.max('home_size'),
                   'popular_area': vaex.agg.count('sold_price'),
                   'sold_above_list_price': vaex.agg.sum('earn_passive'),
                   'sold_below_list_price': vaex.agg.sum('earn_negative')
              }
          )
df_res1
#df_res1.export_csv('res1.csv')

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,sold_above_list_price,sold_below_list_price
0,M2M,2019,house,1658787.7982608695,1675520.9008695653,14.23076923076923,12.415458937198068,53000.0,65000.0,24.0,575,151,424
1,M2M,2019,condo,511935.7828348504,504970.0390117035,14.928205128205128,5.441322314049587,251000.0,229000.0,10.0,769,247,522
2,M2M,2020,house,1497870.2211538462,1514043.5692307693,22.0,12.149606299212598,209100.0,212900.0,22.0,130,40,90
3,M2M,2020,condo,345494.90379008744,347223.0583090379,17.944,5.445427728613569,135000.0,139900.0,8.0,343,60,283
4,L4L,2019,house,1055380.8172161172,1074575.931135531,17.629545454545454,13.055855855855857,390000.0,345000.0,41.0,1365,314,1051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2,078",LC7,2020,house,1215000.0,1199900.0,1.0,13.0,1215000.0,1199900.0,13.0,1,1,0
"2,079",M65,2020,condo,640000.0,649000.0,21.0,6.0,640000.0,649000.0,6.0,1,0,1
"2,080",LOG,2020,house,745000.0,763950.0,10.0,12.0,675000.0,698900.0,16.0,2,0,2
"2,081",L75,2020,house,220000.0,227900.0,nan,6.0,220000.0,227900.0,6.0,1,0,1


In [392]:
    # find specific place of data, please change the 3 character in first_three_postal
first_three_postal = 'M2M'
target_place = df_res1[df_res1['region_place'] == first_three_postal]
target_place

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,sold_above_list_price,sold_below_list_price
0,M2M,2019,house,1.65879e+06,1.67552e+06,14.2308,12.4155,53000,65000,24,575,151,424
1,M2M,2019,condo,511936,504970,14.9282,5.44132,251000,229000,10,769,247,522
2,M2M,2020,house,1.49787e+06,1.51404e+06,22,12.1496,209100,212900,22,130,40,90
3,M2M,2020,condo,345495,347223,17.944,5.44543,135000,139900,8,343,60,283


In [393]:
    # find the avg of popular deals
    # it means how many houses were sold
mean_pop = df_res1['popular_area'].mean()

df_res1['is_popular'] = df_res1['popular_area'] >= 200
df_res1

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,sold_above_list_price,sold_below_list_price,is_popular
0,M2M,2019,house,1658787.7982608695,1675520.9008695653,14.23076923076923,12.415458937198068,53000.0,65000.0,24.0,575,151,424,True
1,M2M,2019,condo,511935.7828348504,504970.0390117035,14.928205128205128,5.441322314049587,251000.0,229000.0,10.0,769,247,522,True
2,M2M,2020,house,1497870.2211538462,1514043.5692307693,22.0,12.149606299212598,209100.0,212900.0,22.0,130,40,90,False
3,M2M,2020,condo,345494.90379008744,347223.0583090379,17.944,5.445427728613569,135000.0,139900.0,8.0,343,60,283,True
4,L4L,2019,house,1055380.8172161172,1074575.931135531,17.629545454545454,13.055855855855857,390000.0,345000.0,41.0,1365,314,1051,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2,078",LC7,2020,house,1215000.0,1199900.0,1.0,13.0,1215000.0,1199900.0,13.0,1,1,0,False
"2,079",M65,2020,condo,640000.0,649000.0,21.0,6.0,640000.0,649000.0,6.0,1,0,1,False
"2,080",LOG,2020,house,745000.0,763950.0,10.0,12.0,675000.0,698900.0,16.0,2,0,2,False
"2,081",L75,2020,house,220000.0,227900.0,nan,6.0,220000.0,227900.0,6.0,1,0,1,False


In [394]:
    # find the avg of ranks
    # it means how many avg ranks of the place
total_mean_rank = df_res1['mean_rank'].mean()
total_mean_rank
df_res1['good_rank'] = df_res1['mean_rank'] >= 18
#df_res1
    # besty ranking
#total_mean_rank = df_res1['mean_rank'].max()

In [395]:
    # please change the value you want here
    # changing values here to make further method output different results
# mainly with 2019 and 2020
target_year = '2019'
# house deal avg which is higher than the mean
target_popular = True
# two types: house or condo
target_house_type = 'condo'
# do the ranking higher than the mean
is_good_rank = True
# tpye a size for searching
target_size = ''


In [397]:
# lowest mean sold place
# find cheapest place

# choose a year
lowest_sold_mean = df_res1[(df_res1['year'] == target_year)]
# want popular place or not
lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['is_popular'] == target_popular)]
# rank good or not
lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['good_rank'] == is_good_rank)]
# house or condo
lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['home_type'] == target_house_type)]

# home size
#lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['mean_size'] >= target_size)]

# find lowest mean place
lowest_sold_mean = lowest_sold_mean[lowest_sold_mean['mean_sold'] == lowest_sold_mean['mean_sold'].min()]

lowest_sold_mean

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,sold_above_list_price,sold_below_list_price,is_popular,good_rank
0,L9R,2019,condo,517610,525411,18.2747,7.99522,222000,223900,16,248,35,213,True,True


In [ ]:
#################################################################################################################



#      no need to check the code below




#################################################################################################################

In [ ]:
# place with large size
size_home = df_res1[(df_res1['year'] == target_year)]
lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['is_popular'] == target_popular)]
lowest_sold_mean = lowest_sold_mean[(lowest_sold_mean['home_type'] == target_house_type)]
lowest_sold_mean = lowest_sold_mean[lowest_sold_mean['mean_sold'] == lowest_sold_mean['mean_sold'].min()]
lowest_sold_mean

In [294]:
    # lowest mean sold place 2019
place_lowest_mean_deals_2019 = df_res1[(df_res1['year'] == '2019')]
place_lowest_mean_deals_2019 = place_lowest_mean_deals_2019[place_lowest_mean_deals_2019['mean_sold'] == place_lowest_mean_deals_2019['mean_sold'].min()]
place_lowest_mean_deals_2019

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money,is_popular
0,V0N,2019,condo,57000,60000,14,nan,57000,60000,inf,1,0,1,False


In [295]:
    # lowest mean sold place 2020
place_lowest_mean_deals_2020 = df_res1[(df_res1['year'] == '2020')]
place_lowest_mean_deals_2020 = place_lowest_mean_deals_2020[place_lowest_mean_deals_2020['mean_sold'] == place_lowest_mean_deals_2020['mean_sold'].min()]
place_lowest_mean_deals_2020

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money,is_popular
0,N0P,2020,condo,58000,65000,1,5,58000,65000,5,1,0,1,False


In [298]:
    # lowest mean sold place 2019 in popular area
pop_place_lowest_mean_deals_2019 = df_res1[(df_res1['year'] == '2019')]
pop_place_lowest_mean_deals_2019 = pop_place_lowest_mean_deals_2019[(pop_place_lowest_mean_deals_2019['is_popular'] == True)]
pop_place_lowest_mean_deals_2019 = pop_place_lowest_mean_deals_2019[pop_place_lowest_mean_deals_2019['mean_sold'] == pop_place_lowest_mean_deals_2019['mean_sold'].min()]
pop_place_lowest_mean_deals_2019

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money,is_popular
0,M9R,2019,condo,267699,272955,12.4202,5.52281,85000,87500,4,340,107,233,True


In [299]:
    # lowest mean sold place 2020 in popular area
pop_place_lowest_mean_deals_2020 = df_res1[(df_res1['year'] == '2020')]
pop_place_lowest_mean_deals_2020 = pop_place_lowest_mean_deals_2020[(pop_place_lowest_mean_deals_2020['is_popular'] == True)]
pop_place_lowest_mean_deals_2020 = pop_place_lowest_mean_deals_2020[pop_place_lowest_mean_deals_2020['mean_sold'] == pop_place_lowest_mean_deals_2020['mean_sold'].min()]
pop_place_lowest_mean_deals_2020

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money,is_popular
0,M9R,2020,condo,175963,180820,18.3279,5.22925,43500,79000,3,257,34,223,True


In [300]:
    # lowest mean sold house 2019
house_lowest_mean_deals_2019 = df_res1[(df_res1['year'] == '2019')]
house_lowest_mean_deals_2019 = house_lowest_mean_deals_2019[(house_lowest_mean_deals_2019['home_type'] == 'house')]
house_lowest_mean_deals_2019 = house_lowest_mean_deals_2019[house_lowest_mean_deals_2019['mean_sold'] == house_lowest_mean_deals_2019['mean_sold'].min()]
house_lowest_mean_deals_2019

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money,is_popular
0,C0K,2019,house,80000,88000,nan,nan,80000,88000,inf,1,0,1,False


In [ ]:
    # lowest mean sold house 2020
house_lowest_mean_deals_2020 = df_res1[(df_res1['year'] == '2020')]
house_lowest_mean_deals_2020 = house_lowest_mean_deals_2020[(house_lowest_mean_deals_2020['home_type'] == 'house')]
house_lowest_mean_deals_2020 = house_lowest_mean_deals_2020[house_lowest_mean_deals_2020['mean_sold'] == house_lowest_mean_deals_2020['mean_sold'].min()]
house_lowest_mean_deals_2020

In [244]:
    # lowest mean sold condo 2019
condo_lowest_mean_deals_2019 = df_res1[(df_res1['year'] == '2019')]
condo_lowest_mean_deals_2019 = condo_lowest_mean_deals_2019[(condo_lowest_mean_deals_2019['home_type'] == 'condo')]
condo_lowest_mean_deals_2019 = condo_lowest_mean_deals_2019[condo_lowest_mean_deals_2019['mean_sold'] == condo_lowest_mean_deals_2019['mean_sold'].min()]
condo_lowest_mean_deals_2019

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money
0,N0P,2020,condo,58000,65000,1,5,58000,65000,5,1,0,1


In [245]:
    # lowest mean sold condo 2020
condo_lowest_mean_deals_2020 = df_res1[(df_res1['year'] == '2020')]
condo_lowest_mean_deals_2020 = condo_lowest_mean_deals_2020[(condo_lowest_mean_deals_2020['home_type'] == 'condo')]
condo_lowest_mean_deals_2020 = condo_lowest_mean_deals_2020[condo_lowest_mean_deals_2020['mean_sold'] == condo_lowest_mean_deals_2020['mean_sold'].min()]
condo_lowest_mean_deals_2020

#,region_place,year,home_type,mean_sold,mean_price,mean_rank,mean_size,min_sold,min_price,max_size,popular_area,make_money,lose_money
0,V0N,2019,condo,57000,60000,14,nan,57000,60000,inf,1,0,1


In [246]:
place_best_mean_deals_2019 = df_res1[(df_res1['year'] == '2019')]
place_best_mean_deals_2019 = place_best_deals_2019[place_best_deals_2019['mean_sold'] == place_best_deals_2019['mean_sold'].max()]
place_best_mean_deals_2019

#,region_place,year,home_type,mean_sold,mean_price,make_money,lose_money
0,M4W,2019,house,3.4539e+06,3.4769e+06,106,131


In [247]:
place_best_mean_deals_2020 = df_res1[(df_res1['year'] == '2020')]
place_best_mean_deals_2020 = place_best_deals_2020[place_best_deals_2020['mean_sold'] == place_best_deals_2020['mean_sold'].max()]
place_best_mean_deals_2020

#,region_place,year,home_type,mean_sold,mean_price,make_money,lose_money
0,M5P,2020,house,3.62651e+06,3.73924e+06,20,40


In [248]:
condo_best_deals_2019 = df_res1[(df_res1['year'] == '2019') and (df_res1['mean_sold'].max == True) and  (df_res1['home_type'] == 'condo')]
condo_best_deals_2019

['M2M',
 '2019',
 'house',
 1658787.7982608695,
 1675520.9008695653,
 14.23076923076923,
 12.415458937198068,
 53000.0,
 65000.0,
 4.0,
 575,
 163,
 412]

In [1011]:
df_sell_20 = df_sell[df_sell['year'] == '2020']
df_sell_19 = df_sell[df_sell['year'] == '2019']


df_sell_20_sep = df_sell[(df_sell['year'] == '2020') & (df_sell['month'] == '09')]
df_sell_19_sep = df_sell[(df_sell['year'] == '2019') & (df_sell['month'] == '09')]
#df_sell_20 = df_sell[df_sell['updated_on'].str.contains('2020-')]
#df_sell_19 = df_sell[df_sell['updated_on'].str.contains('2019-')]

#df_sell_sep_19 = df_sell[df_sell['year'] == '2019' and df_sell['month'] == '09']
#df_sell_sep_20 = df_sell[df_sell['year'] == '2020' and df_sell['month'] == '09']

#df_testres = df_sell[df_sell['year'].str.len()<4]

In [966]:
df_sell_19_sep

#,address,municipality,province,postal_code,price,description,bedrooms,bathrooms,status_id,style_id,created_at,updated_on,extras,address_components,parking_spaces,has_garage,has_locker,url_slug,square_footage,kitchen_count,is_rental,expired_at,sold_price,sold_at,thumbnails_count,address_id,address_ids,time,year,month,municipality_same,province_same,region_territory,region_place,condo,house,earn,earn_passive,earn_negative
0,"113 Meadowview Blvd, Clarington, ON, L1C4X4",Clarington,ON,L1C4X4,660000.0,'Truly One Of A Kind! This Charming 4 + 1 Bedroo...,4.0,4.0,3,101,2019-07-25 11:54:25.195565-04,2019-09-15 17:49:21.662694,"'{""lot"": {""unit"": ""feet"", ""depth"": 114.96, ""fron...","'{""postal"":""L1C4X4"",""street"":""113 Meadowview Blv...",4.0,t,f,113-meadowview-blvd-clarington-on,"[2000,2501)",1.0,f,2019-09-12 00:00:00-04,640000.0,None,20.0,371913.0,"{2794,1,2,7218}",2019-09,2019,09,clarington,on,L,L1C,False,True,-20000.0,False,True
1,"'15 Pine Vista Ave, Whitchurch-Stouffville, ON, ...",Whitchurch-Stouffville,ON,L4A1R9,1780000.0,"'4,100 Sf Custom-Built Bungalow Loft On A 1-Acre...",3.0,4.0,3,101,2019-05-17 19:32:43.630743-04,2019-09-15 17:49:27.567509,"'{""lot"": {""unit"": ""feet"", ""depth"": 268.92, ""fron...","'{""postal"":""L4A1R9"",""street"":""15 Pine Vista Ave""...",15.0,t,f,15-pine-vista-ave-whitchurch-stouffville-on,"[3500,5001)",1.0,f,2019-09-13 00:00:00-04,1675000.0,None,19.0,865368.0,"{1,2,2867}",2019-09,2019,09,whitchurch-stouffville,on,L,L4A,False,True,-105000.0,False,True
2,"1708-231 Fort York Blvd, Toronto, ON, M5V1B2",Toronto,ON,M5V1B2,499900.0,'Spectacular View From This 17th Floor Suite. Th...,1.0,1.0,3,105,2019-07-10 14:50:05.123662-04,2019-09-07 02:43:32.57975,"'{""tax"": {""year"": 2019, ""amount"": 1547.68}, ""pet...","'{""unit"":""1708"",""postal"":""M5V1B2"",""street"":""231 ...",nan,f,f,1708-231-fort-york-blvd-toronto-on,"[500,600)",1.0,f,2019-08-13 00:00:00-04,470000.0,None,6.0,1356199.0,"{1,2,2875,8181,12826}",2019-09,2019,09,toronto,on,M,M5V,True,False,-29900.0,False,True
3,"317 Whitby Shores Green Way, Whitby, ON, L1N9N6",Whitby,ON,L1N9N6,1175000.0,'An Opportunity To Own This Gorgeous Home In The...,4.0,3.0,3,101,2019-07-24 17:47:07.610775-04,2019-09-07 05:32:56.120124,"'{""lot"": {""unit"": ""feet"", ""depth"": 111.55, ""fron...","'{""postal"":""L1N9N6"",""street"":""317 Whitby Shores ...",4.0,t,f,317-whitby-shores-green-way-whitby-on,,1.0,f,2019-08-12 00:00:00-04,1143000.0,None,20.0,1281640.0,"{2676,1,2,9339}",2019-09,2019,09,whitby,on,L,L1N,False,True,-32000.0,False,True
4,"617-5 Rosehill Ave, Toronto, ON, M4T3A6",Toronto,ON,M4T3A6,599300.0,"'Astonishing Two Bedroom, Two Full Bathroom In T...",2.0,2.0,3,105,2019-07-11 20:39:30.627672-04,2019-09-07 16:01:17.946827,"'{""tax"": {""year"": 2019, ""amount"": 2586.64}, ""pet...","'{""unit"":""617"",""postal"":""M4T3A6"",""street"":""5 Ros...",1.0,t,t,617-5-rosehill-ave-toronto-on,None,1.0,f,2019-07-17 00:00:00-04,787000.0,None,20.0,1312570.0,"{1,2,2875,8199,39747}",2019-09,2019,09,toronto,on,M,M4T,True,False,187700.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"20,556","86 Evershot Cres, Markham, ON, L6E0L7",Markham,ON,L6E0L7,839900.0,'Detached Home In High Demand Wismer Community*F...,3.0,4.0,3,101,2019-06-17 11:45:10.684779-04,2019-09-05 23:27:00.265532,"'{""lot"": {""unit"": ""feet"", ""depth"": 98.43, ""front...","'{""postal"":""L6E0L7"",""street"":""86 Evershot Cres"",...",3.0,t,f,86-evershot-cres-markham-on,,1.0,f,2019-07-11 00:00:00-04,829900.0,None,1.0,163819.0,"{2865,1,2,7558}",2019-09,2019,09,markham,on,L,L6E,False,True,-10000.0,False,True
"20,557","17 Munch Pl, Milton, ON, L9T8K6",Milton,ON,L9T8K6,689900.0,"'Spectacular Sought After Area & ""Iris"" Model Ta...",3.0,3.0,3,102,2019-06-22 15:39:16.908921-04,2019-09-05 14:59:18.144603,"'{""lot"": {""unit"": ""feet"", ""depth"": 105, ""front"":...","'{""postal"":""L9